In [2]:
# 简单使用
import numpy as np
import tensorflow as tf
tf.compat.v1.reset_default_graph()
var1=tf.Variable(1.)

decay=0.99
ema=tf.train.ExponentialMovingAverage(decay=decay)
print("对var1进行apply，相当于告诉ema需要维护关于var1的滑动平均值")
ema.apply([var1])
print("\n首次apply，会直接赋值，此时结果为1")
print(ema.average(var1).numpy())
print("\n修改var1为10")
var1.assign(10)
print(var1)
print("\n对var1进行apply，相当于更新关于var1的滑动平均值")
ema.apply([var1])

print("\n求取滑动平均值的方法是使用average，传入的参数就是要求关于谁的滑动平均值")
print(ema.average(var1).numpy())
print("\n计算方式为")
print(1*decay+(1-decay)*10)

对var1进行apply，相当于告诉ema需要维护关于var1的滑动平均值

首次apply，会直接赋值，此时结果为1
1.0

修改var1为10
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=10.0>

对var1进行apply，相当于更新关于var1的滑动平均值

求取滑动平均值的方法是使用average，传入的参数就是要求关于谁的滑动平均值
1.0899999

计算方式为
1.09


In [68]:
# tf2是object-based的，这影响了ema的apply和average方法

import numpy as np
import tensorflow as tf
tf.compat.v1.reset_default_graph()
var1=tf.Variable(1.)
var2=tf.Variable(10.)
var3=tf.Variable(20.)

decay=0.99
ema=tf.train.ExponentialMovingAverage(decay=decay)
print("\napply方法输入两个变量，那么这个ema就会维护关于var1和var2的两个滑动平均值")
ema.apply([var1,var2])
print("\n首次apply，会直接赋值，此时结果为1和10")
print(ema.average(var1).numpy())
print(ema.average(var2).numpy())
print("\n只修改var1为10，并只对var1进行apply，仅仅更新了var1的滑动平均值")
var1.assign(10)
ema.apply([var1])

print("\n求取滑动平均值")
print("\n求取var1的滑动平均值，发现关于var1的滑动平均值已经更新")
print(ema.average(var1).numpy())
print("\n求取var2的滑动平均值，发现关于var2的滑动平均值还是10")
print(ema.average(var2).numpy())
print("\n求取var3的滑动平均值，发现ema并没有维护关于var3的滑动平均值，因此返回None")
print(ema.average(var3))


apply方法输入两个变量，那么这个ema就会维护关于var1和var2的两个滑动平均值

首次apply，会直接赋值，此时结果为1和10
1.0
10.0

只修改var1为10，并只对var1进行apply，仅仅更新了var1的滑动平均值

求取滑动平均值

求取var1的滑动平均值，发现关于var1的滑动平均值已经更新
1.0899999

求取var2的滑动平均值，发现关于var2的滑动平均值还是10
10.0
None


In [100]:
# num_updates参数
import tensorflow as tf
import numpy as np
tf.compat.v1.reset_default_graph()
var1=tf.Variable(1.)

decay=0.99
num_updates=tf.Variable(10.)
ema=tf.train.ExponentialMovingAverage(decay=decay,num_updates=num_updates)
print("对var1进行apply，相当于告诉ema需要维护关于var1的滑动平均值")
ema.apply([var1])
moving_mean=ema.average(var1).numpy() # 此处要取numpy，否则不取numpy的话返回的是variable，后面的值会改变
var1.assign(10)
ema.apply([var1])
print("\n求取滑动平均值的方法是使用average，传入的参数就是要求关于谁的滑动平均值")
print(ema.average(var1).numpy())
print("手动计算")
real_decay=tf.math.minimum(decay,(1.+num_updates)/(10.+num_updates))
moving_mean=(moving_mean*real_decay+(1-real_decay)*var1).numpy()
print(moving_mean)

print("\n修改num_updates值为20，该值会在下一次apply时生效")
num_updates.assign(20)
ema.apply([var1])
real_decay=tf.math.minimum(decay,(1.+num_updates)/(10.+num_updates))
print("求取滑动平均值")
print(ema.average(var1).numpy())
print("手动计算")
moving_mean=(moving_mean*real_decay+(1-real_decay)*var1).numpy()
print(moving_mean)

"""
结论，num_updates可以被赋值为global_step，保证初期的时候real_decay更大，后期的时候使用输入的decay
"""

对var1进行apply，相当于告诉ema需要维护关于var1的滑动平均值

求取滑动平均值的方法是使用average，传入的参数就是要求关于谁的滑动平均值
5.0499997
手动计算
5.05

修改num_updates值为20，该值会在下一次apply时生效
求取滑动平均值
6.535
手动计算
6.535


'\n结论，num_updates可以被赋值为global_step，保证初期的时候real_decay更大，后期的时候使用输入的decay\n'

In [101]:
class C:
    def __init__(self,a):
        self.a=a
def high(a):
    c=C(a)
    def low():
        c.a=c.a+10
        return c.a
    return low

low=high(10)
print(low())
print(low())

    


20
30
